In [11]:
import os
import re
import pandas as pd

# === CONFIG ===
REPORT_DIR = r"F:\Code\random-plotter\run\10"
OUTPUT_CSV = os.path.join("data", "randomSearchv3", "csv", "detailed_summary_report_10.csv")

# Keys to extract from filename
filename_keys = ["Agent_movementModel", "Target_movementModel", "LevyFlight.alpha"]

# === HELPERS ===

def parse_filename_metadata(filename, keys_to_extract):
    base = os.path.splitext(filename)[0]
    metadata = {'filename': base}
    for key in keys_to_extract:
        pattern = rf'{re.escape(key)}@([^\-_@]+)'
        match = re.search(pattern, base)
        metadata[key] = match.group(1) if match else ''
    return metadata

def parse_csv_file(filepath):
    try:
        df = pd.read_csv(filepath)
        if df.empty or "percentage" not in df.columns:
            return None

        result = {
            "min_coverage": df["percentage"].min(),
            "avg_coverage": df["percentage"].mean(),
            "max_coverage": df["percentage"].max()
        }

        for target in [25, 50, 75, 90]:
            time_reached = df[df["percentage"] >= target]
            result[f"time_to_{target}_percent"] = time_reached["time"].iloc[0] if not time_reached.empty else 0

        return result
    except Exception as e:
        print(f"❌ Failed to process {filepath}: {e}")
        return None

# === MAIN EXTRACTION ===

def extract_simulation_summaries():
    os.makedirs(os.path.dirname(OUTPUT_CSV), exist_ok=True)
    all_data = []

    for filename in os.listdir(REPORT_DIR):
        if filename.endswith('.csv') and "SearchingCoverageOverTimeReport" in filename:
            filepath = os.path.join(REPORT_DIR, filename)
            metadata = parse_filename_metadata(filename, filename_keys)
            stats = parse_csv_file(filepath)
            if stats is None:
                stats = {
                    "min_coverage": 0,
                    "avg_coverage": 0,
                    "max_coverage": 0,
                    "time_to_25_percent": 0,
                    "time_to_50_percent": 0,
                    "time_to_75_percent": 0,
                    "time_to_90_percent": 0
                }
            all_data.append({**metadata, **stats})

    df = pd.DataFrame(all_data)

    # === Additional Enhancements ===

    # Convert alpha to float if possible for better graphing
    if "LevyFlight.alpha" in df.columns:
        df["LevyFlight.alpha"] = pd.to_numeric(df["LevyFlight.alpha"], errors='coerce')

    # Sort for consistency (optional)
    df = df.sort_values(by=["Agent_movementModel", "Target_movementModel", "LevyFlight.alpha"])

    # Save CSV
    df.to_csv(OUTPUT_CSV, index=False)
    print(f"✅ Extracted summary written to: {OUTPUT_CSV}")
    return df

# Run
extract_simulation_summaries()


✅ Extracted summary written to: data\randomSearchv3\csv\detailed_summary_report_10.csv


,filename,Agent_movementModel,Target_movementModel,LevyFlight.alpha,min_coverage,avg_coverage,max_coverage,time_to_25_percent,time_to_50_percent,time_to_75_percent,time_to_90_percent
0,RandomSearch-Agent_movementModel@BrownianMotio...,BrownianMotion,StationaryClustered,0.25,0.0,39.798817,66.0,154800,410400,0,0
1,RandomSearch-Agent_movementModel@LevyFlight-Ta...,LevyFlight,StationaryClustered,0.25,3.0,67.994083,85.0,39600,133200,244800,0
2,RandomSearch-Agent_movementModel@LevyFlight-Ta...,LevyFlight,StationaryClustered,0.50,1.0,64.360947,91.0,64800,140400,352800,471600
3,RandomSearch-Agent_movementModel@LevyFlight-Ta...,LevyFlight,StationaryClustered,0.75,1.0,69.781065,92.0,68400,129600,266400,388800
4,RandomSearch-Agent_movementModel@LevyFlight-Ta...,LevyFlight,StationaryClustered,1.00,0.0,54.857988,77.0,43200,172800,565200,0
5,RandomSearch-Agent_movementModel@LevyFlight-Ta...,LevyFlight,StationaryClustered,1.25,0.0,46.609467,69.0,86400,262800,0,0
6,RandomSearch-Agent_movementModel@LevyFlight-Ta...,LevyFlight,StationaryClustered,1.50,0.0,27.621302,46.0,277200,0,0,0
7,RandomSearch-Agent_movementModel@LevyFlight-Ta...,LevyFlight,StationaryClustered,1.75,0.0,24.366864,48.0,370800,0,0,0
8,RandomSearch-Agent_movementModel@LevyFlight-Ta...,LevyFlight,StationaryClustered,2.00,0.0,13.585799,30.0,468000,0,0,0
